## Before run
- Please change the selected_label parameter under the **Train the data with all features** heading.
- Change the correlation_threshold under the **Dropping the features using correlation threshold.** heading.

## Load the data

In [63]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from google.colab import drive, files

In [64]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
train_data = pd.read_csv('/content/drive/MyDrive/ML/train.csv')
train_data.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
0,-1.928826,-0.291540,1.105909,2.070652,0.657838,-0.670940,-0.968238,0.049916,1.111149,-1.834592,...,-0.691538,-1.152522,0.441157,-1.494434,-0.344974,-0.395641,45,NaN,1,6
1,-1.709277,-1.008998,0.956611,4.313823,-0.669455,1.928765,0.162550,0.152173,2.356505,-2.898549,...,-0.668235,-0.532695,0.151163,-1.876885,1.651534,-2.706490,45,NaN,1,6
2,-1.906183,-0.357562,-0.682627,4.651838,-0.884940,1.159512,1.354481,-1.432248,2.160144,-3.175735,...,-0.026728,0.440408,2.745906,-0.680754,-0.643588,-2.587203,45,NaN,1,6
3,0.697872,-2.218567,-0.572214,1.192062,0.329554,0.741364,0.962354,0.993512,2.896854,-1.600380,...,0.709032,0.812106,1.055266,-0.915258,-0.924856,-1.318964,45,NaN,1,6
4,-1.976895,-0.496308,0.368102,3.739787,-0.132058,1.234980,-0.501003,0.881554,1.630941,-3.853953,...,-0.599700,-1.024829,1.869753,-2.498570,3.268253,-1.356606,45,NaN,1,6


In [66]:
test_data = pd.read_csv('/content/drive/MyDrive/ML/test.csv')
test_data = test_data.iloc[:,:256]
test_data.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256
0,-1.141206,-0.279703,1.019681,2.605902,0.460391,-1.167380,-1.149165,-1.205164,2.468966,-2.729526,...,1.670388,-0.355608,-0.342747,-0.102831,-0.868282,-0.429783,0.438561,-1.625122,0.599944,-0.615575
1,-1.796767,-0.056758,0.771219,4.414086,-1.229059,1.732098,0.063241,-0.126311,1.322355,-3.461282,...,1.136570,0.826741,-0.371280,-1.492272,0.227806,0.209200,2.200402,-1.701029,1.163999,-1.570136
2,-1.388968,-0.418296,0.186543,2.803852,-1.499095,1.395180,0.739648,0.274060,1.228789,-3.081147,...,0.343188,-0.243809,0.462302,-1.511495,-0.692679,0.449235,1.377531,-1.854203,-0.040538,-2.164104
3,1.276844,-0.112323,-0.310796,2.251175,0.283670,-0.581020,-0.024439,-0.306097,2.424685,-1.714863,...,3.057573,-0.632902,1.511204,-0.658826,-0.690499,-1.077949,0.930102,-2.758352,-0.246738,-1.389013
4,-1.560815,-0.168494,1.249770,1.869219,-1.646049,0.132156,-0.664602,0.483327,1.728020,-2.258071,...,1.217447,-0.374313,1.280207,-1.546299,-0.510597,1.213396,2.497490,-1.597984,-0.163483,-1.735939


In [67]:
train_data.shape

(28520, 260)

Checking for any null values in the data set.

In [68]:
train_data.isnull().sum()

feature_1        0
feature_2        0
feature_3        0
feature_4        0
feature_5        0
              ... 
feature_256      0
label_1          0
label_2        480
label_3          0
label_4          0
Length: 260, dtype: int64

### Droping the data for missing values.

In [69]:
train_data = train_data.dropna()
train_data.isnull().sum()

feature_1      0
feature_2      0
feature_3      0
feature_4      0
feature_5      0
              ..
feature_256    0
label_1        0
label_2        0
label_3        0
label_4        0
Length: 260, dtype: int64

## Train the data with all features.

In [70]:
X_train = train_data.drop(['label_1', 'label_2', 'label_3', 'label_4'], axis=1)
X_train.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256
480,-2.133907,-1.851185,-0.401913,0.579630,0.701037,-0.212690,-0.708243,3.016801,0.464313,-0.825300,...,0.332230,-1.325403,0.563739,-0.180270,-0.598476,2.794869,1.049926,-2.829166,0.908335,-1.295668
481,-0.404385,-0.836757,0.762689,1.637213,0.665621,0.625971,0.262635,1.408128,0.776417,-2.443906,...,-1.451546,-0.435936,-0.216273,-1.313571,1.234601,0.764250,1.848260,-1.138066,-0.143287,-1.997515
482,-1.642496,-1.278682,0.164281,1.371167,0.258172,-0.896479,-0.051402,0.623235,0.367427,-1.211149,...,0.652031,-0.044023,1.547585,-0.239684,0.269733,1.143640,1.827864,-2.056184,-0.070624,-2.222077
483,-0.191927,-2.451684,0.547922,-0.041680,0.804643,-0.324783,0.806728,1.507769,-0.657439,-0.784736,...,-0.799259,-0.702702,0.617131,-0.730333,1.085195,-1.203888,2.832388,-1.943990,0.208316,-2.421425
484,-0.510969,-0.781623,-1.478504,0.794873,0.354653,0.268014,-1.677308,1.627065,0.985870,-2.165971,...,-0.607510,-1.884749,1.849695,0.071822,-0.820887,1.300054,2.499395,-1.692642,0.563708,-0.975423


In [71]:
selected_label = 'label_3' #@param ["label_1", "label_2", "label_3", "label_4"]
y_train = train_data[selected_label]
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_train

array([1, 1, 1, ..., 1, 1, 1])

### Create and Train the XGBoost Model.

In [72]:
# Create an XGBoost classifier (or regressor for regression tasks)
model = xgb.XGBClassifier(verbosity=2, tree_method="gpu_hist")  # You can specify hyperparameters here

# Train the model on the training data
model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

### Loading the valid data set.

In [73]:
valid_data = pd.read_csv("/content/drive/MyDrive/ML/valid.csv")
valid_data.isnull().sum()

feature_1       0
feature_2       0
feature_3       0
feature_4       0
feature_5       0
               ..
feature_256     0
label_1         0
label_2        14
label_3         0
label_4         0
Length: 260, dtype: int64

### Droping data for missing values in the valid dataset.

In [74]:
valid_data = valid_data.dropna()
valid_data.isnull().sum()

feature_1      0
feature_2      0
feature_3      0
feature_4      0
feature_5      0
              ..
feature_256    0
label_1        0
label_2        0
label_3        0
label_4        0
Length: 260, dtype: int64

In [75]:
y_test = valid_data[selected_label]
valid_data = valid_data.iloc[:,:256]
valid_data.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256
14,-0.172181,-2.506089,-0.087602,1.013303,0.838550,-0.172347,-0.543211,1.235440,1.376382,-0.972239,...,-0.853403,-0.489255,0.446155,0.504970,-1.006134,0.184453,0.415477,-1.716953,-0.827742,-2.045459
15,-0.681202,-1.663909,1.788502,0.857073,0.709822,0.834900,0.039072,1.219533,0.483133,-1.896562,...,-2.468510,-0.350192,0.260784,-0.876141,1.675475,0.322208,1.401139,-1.549360,-1.246808,-1.945575
16,-1.634366,0.461024,0.615440,1.199882,0.084413,-1.177391,0.557994,0.975045,0.033425,-0.920055,...,1.033730,0.120623,1.778635,0.175824,-0.532355,0.251711,1.900047,-2.395971,-0.667466,-2.955309
17,-0.963445,-0.215360,-0.653633,0.456256,0.523190,0.178436,0.157641,0.747167,-0.100809,-2.305725,...,0.188098,0.023904,1.613809,0.361577,-0.296968,-0.438388,1.488559,-1.781888,0.081912,-2.422592
18,-0.583766,-1.942095,-0.686590,1.630802,0.921440,0.809160,-0.389961,1.604646,-0.304054,-1.762844,...,0.362820,-0.330955,1.798214,-0.767588,0.749547,-0.493671,2.532719,-1.705593,1.291160,-2.135860


### Make predictions on the valid set

In [76]:
y_pred = model.predict(valid_data)
y_pred = le.inverse_transform(y_pred)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### Make predictions on the test set

In [77]:
y_pred_test = model.predict(test_data)
y_pred_test = le.inverse_transform(y_pred_test)
y_pred_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### Evaluate the model.

In [78]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00


### Adding the predicted values to output dataframe.

In [79]:
output_dataframe = pd.DataFrame()
output_dataframe['Predicted labels before feature engineering'] = y_pred_test
output_dataframe.head()

,Predicted labels before feature engineering
0,1
1,1
2,1
3,1
4,1


## Reducing features using correlation analysis.

### Calculation correlation matrix

In [80]:
correlation_matrix = train_data.corr()
correlation_matrix

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,label_1,label_2,label_3,label_4
feature_1,1.000000,-0.121676,-0.037029,0.145633,0.079656,0.149253,0.028379,-0.147790,0.051300,0.173989,...,0.076297,-0.296220,-0.127254,0.214652,-0.028065,-0.046658,0.204055,-0.054773,0.014545,-0.027671
feature_2,-0.121676,1.000000,-0.317193,-0.039848,-0.066403,0.019331,0.001379,-0.066685,-0.179495,-0.230508,...,-0.098701,0.149414,0.228009,0.141309,0.186994,-0.144739,-0.070000,-0.083949,-0.037420,-0.112897
feature_3,-0.037029,-0.317193,1.000000,0.001755,-0.022552,-0.045518,0.109086,0.003902,0.124212,0.046895,...,0.044539,-0.029690,-0.095992,-0.041498,-0.190645,0.047295,-0.102954,0.122517,0.303640,0.104591
feature_4,0.145633,-0.039848,0.001755,1.000000,0.183209,0.336084,-0.037661,-0.093481,0.130821,-0.083225,...,0.261620,-0.261910,0.031806,0.352850,-0.042392,-0.321713,-0.015159,-0.005010,-0.137331,0.083657
feature_5,0.079656,-0.066403,-0.022552,0.183209,1.000000,0.172993,-0.274253,-0.092687,0.101470,0.087783,...,0.127240,0.043438,0.004587,0.370426,0.250544,0.073181,0.071020,0.068817,0.070020,0.026263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
feature_256,-0.046658,-0.144739,0.047295,-0.321713,0.073181,0.137826,-0.264401,-0.261666,0.212506,-0.240007,...,-0.352998,0.399231,-0.145467,-0.260254,0.180143,1.000000,0.016576,0.021947,0.362926,0.081151
label_1,0.204055,-0.070000,-0.102954,-0.015159,0.071020,-0.096380,0.026733,-0.045493,0.103510,0.128527,...,0.047345,-0.069038,0.022368,0.169159,0.221584,0.016576,1.000000,-0.038758,-0.414031,-0.021261
label_2,-0.054773,-0.083949,0.122517,-0.005010,0.068817,0.089700,-0.092337,-0.090452,0.093125,0.010411,...,-0.008342,0.025528,-0.079649,0.028317,0.015842,0.021947,-0.038758,1.000000,0.086053,0.242613
label_3,0.014545,-0.037420,0.303640,-0.137331,0.070020,0.236802,-0.086687,-0.044259,0.030083,-0.089671,...,-0.079254,0.153056,-0.024106,-0.010441,-0.128754,0.362926,-0.414031,0.086053,1.000000,0.016850


In [81]:
# Calculate the absolute correlations with the target variable
correlations_with_target = correlation_matrix[selected_label].abs()
correlations_with_target.describe()

count    260.000000
mean       0.168963
std        0.149132
min        0.000330
25%        0.065604
50%        0.131144
75%        0.220616
max        1.000000
Name: label_3, dtype: float64

In [82]:
# Sorting the corelations.
sorted_correlations = correlations_with_target.sort_values(ascending=False)
sorted_correlations

label_3        1.000000
feature_233    0.792074
feature_44     0.740370
feature_36     0.697641
feature_113    0.628021
                 ...   
feature_191    0.007772
feature_124    0.006074
feature_22     0.004236
feature_231    0.000622
feature_69     0.000330
Name: label_3, Length: 260, dtype: float64

### Droping the features using correlation threshold.

In [83]:
correlation_threshold = 0.2
low_correlation_mask = (sorted_correlations < correlation_threshold)

# Removing the labels from the correlation mask since value data set don't have them.
labels = ['label_1', 'label_2', 'label_3', 'label_4']
low_correlation_mask_without_labels = low_correlation_mask.drop(labels, axis=0)
low_correlation_mask_without_labels

feature_233    False
feature_44     False
feature_36     False
feature_113    False
feature_143    False
               ...  
feature_191     True
feature_124     True
feature_22      True
feature_231     True
feature_69      True
Name: label_3, Length: 256, dtype: bool

In [84]:
x_train_filtered = X_train.drop(columns=X_train.columns[low_correlation_mask_without_labels])
x_train_filtered.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77
480,-2.133907,-1.851185,-0.401913,0.579630,0.701037,-0.212690,-0.708243,3.016801,0.464313,-0.825300,...,-0.580378,2.452391,2.074651,0.169314,-0.556106,-0.785897,1.360713,-0.564455,1.275188,-0.140501
481,-0.404385,-0.836757,0.762689,1.637213,0.665621,0.625971,0.262635,1.408128,0.776417,-2.443906,...,-0.645075,-2.499217,-0.021338,1.744112,-1.914477,1.416228,1.710147,-0.427184,1.758965,0.129278
482,-1.642496,-1.278682,0.164281,1.371167,0.258172,-0.896479,-0.051402,0.623235,0.367427,-1.211149,...,0.736361,0.592574,-0.370648,-1.166618,-3.109079,-0.898857,0.207523,-0.262288,0.646125,0.932274
483,-0.191927,-2.451684,0.547922,-0.041680,0.804643,-0.324783,0.806728,1.507769,-0.657439,-0.784736,...,-1.263672,-1.450501,0.080405,1.212535,-1.493365,0.059593,1.188014,0.099023,1.577478,1.527307
484,-0.510969,-0.781623,-1.478504,0.794873,0.354653,0.268014,-1.677308,1.627065,0.985870,-2.165971,...,-0.693987,0.444103,1.197832,0.615812,-2.370643,-0.535657,0.930777,-0.945265,2.060421,0.446562


### Create and Train the XGBoost Model.

In [85]:
# Create an XGBoost classifier (or regressor for regression tasks)
model = xgb.XGBClassifier(verbosity=2, tree_method="gpu_hist")  # You can specify hyperparameters here

# Train the model on the training data
model.fit(x_train_filtered, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

### Make predictions on the valid set using the XGBoost model.

In [86]:
valid_data_filtered = valid_data.drop(columns=valid_data.columns[low_correlation_mask_without_labels])

y_pred = model.predict(valid_data_filtered)
y_pred = le.inverse_transform(y_pred)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### Make predictions on the test data set using the XGBoost Model.

In [87]:
test_data_filtered = test_data.drop(columns=test_data.columns[low_correlation_mask_without_labels])

y_pred_test = model.predict(test_data_filtered)
y_pred_test = le.inverse_transform(y_pred_test)
y_pred_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### Evaluate the XGBoost Model.

In [88]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00


### Adding predicted values to output dataframe after feature engineering.

In [89]:
output_dataframe['Predicted labels after feature engineering'] = y_pred_test
output_dataframe['No of new features'] = x_train_filtered.shape[1]
output_dataframe = pd.concat([output_dataframe.reset_index(drop=True), test_data_filtered.reset_index(drop=True)],
                             axis=1)
output_dataframe

,Predicted labels before feature engineering,Predicted labels after feature engineering,No of new features,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77
0,1,1,77,-1.141206,-0.279703,1.019681,2.605902,0.460391,-1.167380,-1.149165,...,0.300957,-0.497910,-0.693194,-0.745785,-1.597484,0.186136,-0.779382,0.857643,0.871580,0.874566
1,1,1,77,-1.796767,-0.056758,0.771219,4.414086,-1.229059,1.732098,0.063241,...,0.109221,-2.619800,0.313859,-0.596263,-2.375077,1.092554,-2.456048,0.696034,0.699237,1.804060
2,1,1,77,-1.388968,-0.418296,0.186543,2.803852,-1.499095,1.395180,0.739648,...,0.248497,-2.975638,-0.403364,1.368735,-1.394819,0.995371,-0.753391,-0.289487,0.494453,-0.924538
3,1,1,77,1.276844,-0.112323,-0.310796,2.251175,0.283670,-0.581020,-0.024439,...,2.632051,-1.349739,0.570638,0.165003,-1.766913,-1.185904,-1.651145,0.682192,1.581084,-0.258739
4,1,1,77,-1.560815,-0.168494,1.249770,1.869219,-1.646049,0.132156,-0.664602,...,0.842021,-0.414934,0.349068,-0.787115,-2.375710,0.351577,-2.202368,-0.136861,0.158608,-0.396097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,1,1,77,-0.520450,-1.231046,0.823668,-0.786737,-0.544497,-1.154617,0.334333,...,-1.289042,-0.682407,0.537221,1.121163,-2.292451,-0.764069,1.631783,-0.121476,0.652998,0.801760
746,1,1,77,-0.512158,-2.199031,2.003965,-1.231407,-0.056881,-0.851074,0.856941,...,-0.511170,2.007282,1.873870,-0.391854,-2.586747,0.093436,0.925082,-1.509342,0.433631,-0.985240
747,1,1,77,0.160726,-1.636096,-1.372266,1.134842,0.232438,0.352374,-1.142442,...,2.218376,-1.358147,1.477038,0.405914,-3.341241,-1.185824,1.062763,-0.042336,1.408227,0.762650
748,1,1,77,0.546223,-0.257980,2.767929,0.483799,1.346494,-0.963523,-0.712168,...,-1.000145,-2.345668,0.106439,-0.989255,-1.908630,-2.606727,0.604310,-0.917566,1.420674,0.557965


### Exporting the dataframe to csv.

In [90]:
filename = '190145G_'+selected_label+'.csv'

# Adding null columns
start_point = output_dataframe['No of new features'][0]
output_dataframe = output_dataframe.assign(**{f'New_Column_{i}': None for i in range(start_point, 256)})

# Renaming the columns as needed.
new_column_names = ['Predicted labels before feature engineering', 'Predicted labels after feature engineering', 'No of new features']
for i in range(1, 257):
  new_column_names.append('new_feature_'+str(i))

output_dataframe.columns = new_column_names
output_dataframe.to_csv(filename)
files.download(filename)

<ipython-input-90-176b6b2d2c5c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_dataframe = output_dataframe.assign(**{f'New_Column_{i}': None for i in range(start_point, 256)})
<ipython-input-90-176b6b2d2c5c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output_dataframe = output_dataframe.assign(**{f'New_Column_{i}': None for i in range(start_point, 256)})
<ipython-input-90-176b6b2d2c5c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Create and Train the SVC Model.

In [91]:
svc_model = SVC(kernel='linear', C=1.0, random_state=42)
svc_model.fit(x_train_filtered, y_train)

SVC(kernel='linear', random_state=42)

### Make predictions on the test set using the SVC Model.

In [92]:
y_pred = svc_model.predict(valid_data_filtered)
y_pred = le.inverse_transform(y_pred)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

### Evaluate the SVC Model.

In [93]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00


### Conculusion
#### Label 1
- After reducing the features to 176 still we can achieve the **0.94** accuracy using the XGBoost model with the correlation threshold 0.05.
- Since the filtering already shown some performance I tried the dataset on the SVC Model and it gives the **0.999** accuracy.
- SVC model is more suited for this dataset.

#### Label 2
- Reduced the features to 127 and it has **0.93** accuracy on XGBoost model with the correlation threshold 0.05.

#### Label 3
- Reduced the features to 77 and it has **1.00** accuracy on XGBoost model with the correlation threshold 0.2.

#### Label 4
- Reduced the features to 117 and it has **0.94** accuracy on XGBoost model with the correlation threshold 0.05.
